In [1]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm, trange
from data.data import SphericalDataset, SphericalBlockDataset, stand_norm
from data.vis_io import get_mesh

from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline, Pipeline
from cfg.configs import Config
from util.utils import count_parameters
from train_block import resume_checkpoint_block, load_checkpoint_block
from test_block import reconstruct_blocks
from network.network import MLP
%load_ext autoreload
%autoreload 2

nncfg = Config("nn_block_half.yaml")
nncfg = Config("nn_single_block.yaml")
testcfg = Config("nntest_block_half.yaml")

In [50]:
model = nncfg.get_model()

In [54]:
a[..,None].shape

torch.Size([1, 3, 5])

In [52]:
a = torch.rand(3, 5)

In [4]:
grid_dataset = SphericalBlockDataset(**nncfg.get_dataset_param())

transforming inputs
transforming targets
creating train test splits for inputs
creating train test splits for targets


In [17]:
num_block = len(grid_dataset)
comps = []
for bi in tqdm(range(num_block)):
  phys = grid_dataset.curvs[bi][0]
  phys_trans = grid_dataset.curvs[bi][1]
  comp = grid_dataset.carts[bi][0]
  comp_trans = grid_dataset.carts[bi][1]
  comps.append(comp)

100%|██████████| 8/8 [00:00<00:00, 72471.78it/s]


In [20]:
grid_dataset.curv.shape

torch.Size([201, 360, 160, 3])

In [29]:
b1xdim, b1ydim, b1zdim = grid_dataset.block_indices[0]
curv_b1 = grid_dataset.curv[b1xdim[0]:b1xdim[1], b1ydim[0]:b1ydim[1], b1zdim[0]:b1zdim[1]]
cart_b1 = grid_dataset.cart[b1xdim[0]:b1xdim[1], b1ydim[0]:b1ydim[1], b1zdim[0]:b1zdim[1]]


In [30]:
curv_b1.shape

torch.Size([100, 180, 80, 3])

In [31]:
cart_b1.shape

torch.Size([100, 180, 80, 3])

In [35]:
earth_nopole_block = np.concatenate([curv_b1, cart_b1], axis=-1)
np.save("data/earth_nopole_block.npy", earth_nopole_block)

In [24]:
comps[7].reshape(100, 180, 80, 3).shape

(100, 180, 80, 3)

In [6]:
eval_data = torch.load("eval/phys2comp_eval.pk")
comp_preds = eval_data['comp_preds']

In [9]:
comp = grid_dataset.curv

In [15]:
comp.reshape(-1, 3).shape

torch.Size([11577600, 3])

In [13]:
comp_pred = np.concatenate(comp_preds, axis=0).reshape(comp.shape)

ValueError: cannot reshape array of size 34560000 into shape (201,360,160,3)

In [14]:
np.concatenate(comp_preds, axis=0).shape

(11520000, 3)